In [1]:
!pip install dash
!pip install dash_core_components
!pip install dash_html_components

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import dash_bootstrap_components as dbc

C:\Users\vansh\AppData\Local\Temp/ipykernel_5976/1629002191.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\vansh\AppData\Local\Temp/ipykernel_5976/1629002191.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
import json
import folium
import requests
import mimetypes
import http.client
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

conn= http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers={}
conn.request("GET","/summary",payload,headers)
res=conn.getresponse()
data=res.read().decode('UTF-8')
pd.set_option("display.max_rows", None, "display.max_columns", None)
covid1=json.loads(data)
pd.json_normalize(covid1['Countries'],sep=",")
df=pd.DataFrame(covid1['Countries'])
covid2=df.drop(columns=['ID','CountryCode','Slug','Date','Premium'],axis=1)
covid2=covid2.sort_values(by=['TotalConfirmed'],ascending=False)
covid2['Country'] = covid2['Country'].replace('Russian Federation', 'Russia')
covid2['Country'] = covid2['Country'].replace('Iran, Islamic Republic of', 'Iran')
covid2['Country'] = covid2['Country'].replace('Venezuela (Bolivarian Republic)', 'Venezuela')
covid3=covid2
# SARS
sars_03 = pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\sars_2003_complete_dataset_clean.csv",parse_dates=['Date'])
sars_03 = sars_03[['Date', 'Country', 'Cumulative number of case(s)','Number of deaths', 'Number recovered']]
sars_03.columns = ['Date', 'Country', 'Cases', 'Deaths', 'Recovered']
sars_03 = sars_03.groupby(['Date', 'Country','Cases', 'Deaths', 'Recovered'])
sars_03 = sars_03.sum().reset_index()
sars_03['Cases'] = sars_03['Cases'].fillna(0)
sars_03['Deaths'] = sars_03['Deaths'].fillna(0)
sars_03['Recovered'] = sars_03['Recovered'].fillna(0)
sars_latest = sars_03[sars_03['Date'] == max(sars_03['Date'])].reset_index()
sars_latest=sars_latest.sort_values(by=['Cases'],ascending=False)

# H1N1
h1n1_09 = pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\Pandemic_H1N1_2009.csv",parse_dates=['Update Time'])
h1n1_09 = h1n1_09[['Update Time', 'Country', 'Cases','Deaths']]
h1n1_09 = h1n1_09.groupby(['Update Time', 'Country','Cases', 'Deaths'])
h1n1_09 = h1n1_09.sum().reset_index()
h1n1_09['Cases'] = h1n1_09['Cases'].fillna(0)
h1n1_09['Deaths'] = h1n1_09['Deaths'].fillna(0)
h1n1_09['Deaths'] = h1n1_09['Deaths'].astype('int')
h1n1_latest=h1n1_09[h1n1_09['Update Time'] == max(h1n1_09['Update Time'])].reset_index()
h1n1_latest=h1n1_latest.sort_values(by=['Cases'],ascending=False)
h1n1_latest=h1n1_latest.drop([43])

# EBOLA
ebola_14 = pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\ebola_2014_2016_clean.csv",parse_dates=['Date'])
ebola_14 = ebola_14[['Date', 'Country',
 'Cumulative no. of confirmed, probable and suspected cases','Cumulative no. of confirmed, probable and suspected deaths']]
ebola_14.columns = ['Date', 'Country', 'Cases', 'Deaths']
ebola_14 = ebola_14.groupby(['Date', 'Country','Cases', 'Deaths'])
ebola_14 = ebola_14.sum().reset_index()
ebola_14['Cases'] = ebola_14['Cases'].fillna(0)
ebola_14['Deaths'] = ebola_14['Deaths'].fillna(0)
ebola_14['Cases'] = ebola_14['Cases'].astype('int')
ebola_14['Deaths'] = ebola_14['Deaths'].astype('int')
ebola_latest = ebola_14[ebola_14['Date'] == max(ebola_14['Date'])].reset_index()
ebola_latest=ebola_latest.sort_values(by=['Cases'],ascending=False)

#MERS
mers_latest = pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\country_count_latest.csv")
mers_latest=mers_latest.sort_values(by=['Confirmed'],ascending=False)

In [4]:
fig = px.treemap(covid3, names='Country', parents=[""]*len(covid3), values='TotalConfirmed')
fig.add_trace(go.Treemap(
    labels = list(covid3.Country),
    parents = [""]*len(covid3),
    values =  list(covid3.TotalConfirmed),
    textinfo = "label+value+percent parent",
    ),row = 1, col = 1)
# fig = plt.figure(figsize=(10, 5))

In [5]:
m=folium.Map(tiles="Stamen Terrain",min_zoom=1.5)
url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes=f'{url}/world-countries.json'
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=covid3,
    columns=['Country','TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='white',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
covid3.update(covid3['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid3.update(covid3['TotalRecovered'].map('Total Recovered:{}'.format))
coordinates=pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\world_country_and_usa_states_latitude_and_longitude_values.csv")
coordinates = coordinates.rename(columns={'country': 'Country'})
coordinates= coordinates.drop(columns=['country_code', 'usa_state_code','usa_state_latitude','usa_state_longitude','usa_state'])
covid_final=pd.merge(covid3,coordinates,on='Country')
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],
                        fill_color='#000000').add_to(m)
covid_final.apply(plotDot,axis=1)
m.fit_bounds(m.get_bounds())


m1=folium.Map(tiles="StamenToner",min_zoom=2)
deaths=covid_final['TotalDeaths'].astype(float)
lat=covid_final['latitude'].astype(float)
lon=covid_final['longitude'].astype(float)
m1=m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.TotalDeaths],
                        fill_color='#000000').add_to(m1)
covid_final.apply(plotDot,axis=1)
m1.fit_bounds(m1.get_bounds())

In [8]:
#app = dash.Dash()
app1 = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H2("Sidebar", className="display-4"),
        html.Hr(),
        html.P(
            "A simple sidebar layout with navigation links", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Home", href="/", active="exact"),
                dbc.NavLink("Page 1", href="/page-1", active="exact"),
                dbc.NavLink("Page 2", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app1.layout = html.Div(children=[
    html.H1(children='Tree Maps'),
#     html.Div([
#         [dcc.Location(id="url"), sidebar, content]
#     ]),
    # All elements from the top of the page
    html.Div([
        html.H3(children='Hello Dash'),

        html.Div(children='''
            Dash: A web application framework for Python.
        '''),

        dcc.Graph(
            id='graph1',
            figure=fig
        ),  
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='Hello Dash'),

        html.Div(children='''
            Dash: A web application framework for Python.
        '''),

        dcc.Graph(
            id='graph2',
            figure=fig
        ),  
    ]),
     # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='Hello Dash'),

        html.Div(children='''
            Dash: A web application framework for Python.
        '''),

        dcc.Graph(
            id='graph3',
            figure=fig
        ),  
    ]),
     # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='Hello Dash'),

        html.Div(children='''
            Dash: A web application framework for Python.
        '''),

        dcc.Graph(
            id='graph4',
            figure=fig
        ),  
    ]),
])
# app.layout = html.Div(children=[
#     html.H1(children='TreeMaps'),
#     html.Div(children='''National Sales Funnel Report.'''),
#     dcc.Graph(
#         id='example-graph',
#         figure=fig)
# ])

if __name__ == '__main__':
    app1.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Dec/2021 18:35:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:35:16] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1632235559.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:35:16] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1632235559.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:35:16] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1632235559.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:35:16] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1632235559.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:35:16] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1632235559.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:35:16] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_0_1m1583589390.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18

In [6]:
"""
This app creates a simple sidebar layout using inline style arguments and the
dbc.Nav component.

dcc.Location is used to track the current location, and a callback uses the
current location to render the appropriate page content. The active prop of
each NavLink is set automatically according to the current pathname. To use
this feature you must install dash-bootstrap-components >= 0.11.0.

For more details on building multi-page Dash applications, check out the Dash
documentation: https://dash.plot.ly/urls
"""
import dash
import dash_bootstrap_components as dbc
from dash import Input, Output, dcc, html

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H2("Sidebar", className="display-4"),
        html.Hr(),
        html.P(
            "A simple sidebar layout with navigation links", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Home", href="/", active="exact"),
                dbc.NavLink("Page 1", href="/page-1", active="exact"),
                dbc.NavLink("Page 2", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([dcc.Location(id="url"), sidebar, content])


@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return html.P("This is the content of the home page!")
    elif pathname == "/page-1":
        return html.P("This is the content of page 1. Yay!")
    elif pathname == "/page-2":
        return html.P("Oh cool, this is page 2!")
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )


if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Dec/2021 18:41:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:41:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:41:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:41:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:41:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:41:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:41:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:41:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:42:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:42:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:42:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 18:42:26] "POST /_dash-update-component

In [ ]:
# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Input, Output
# import pandas as pd
# import plotly.express as px

# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# # assume you have a "long-form" data frame
# # see https://plotly.com/python/px-arguments/ for more options
# df_bar = pd.DataFrame({
#     "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
#     "Amount": [4, 1, 2, 2, 4, 5],
#     "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
# })

# fig = px.bar(df_bar, x="Fruit", y="Amount", color="City", barmode="group")

# app.layout = html.Div(children=[
#     # All elements from the top of the page
#     html.Div([
#         html.H1(children='Hello Dash'),

#         html.Div(children='''
#             Dash: A web application framework for Python.
#         '''),

#         dcc.Graph(
#             id='graph1',
#             figure=fig
#         ),  
#     ]),
#     # New Div for all elements in the new 'row' of the page
#     html.Div([
#         html.H1(children='Hello Dash'),

#         html.Div(children='''
#             Dash: A web application framework for Python.
#         '''),

#         dcc.Graph(
#             id='graph2',
#             figure=fig
#         ),  
#     ]),
# ])

# if __name__ == '__main__':
#     app.run_server(debug=True)